# 📈 VAR: Vector Autoregression для многомерного прогнозирования

Этот блокнот реализует многомерное прогнозирование финансовых временных рядов с использованием **Vector Autoregression (VAR)** модели - классического подхода для анализа взаимосвязанных временных рядов.

## 🎯 Основные особенности

- **Многомерные данные**: используем данные из папки `/data/multivariate_series/`
- **Поэтапные признаки**: различные этапы добавления фичей согласно progressive feature analysis
- **Walk-forward прогнозирование**: честное тестирование с расширяющимся окном
- **Горизонт прогноза**: 10 точек
- **Размер теста**: 11 точек
- **VAR модель**: Vector Autoregression с оптимизацией лагов
- **Метрики**: RMSE, MAPE, DA (Directional Accuracy)

## 📊 Структура этапов признаков

1. **Этап 1**: close + volume (базовые признаки)
2. **Этап 2**: + anomaly
3. **Этап 3**: + weighted_score_with_decay (новости)
4. **Этап 4**: + OHLCV признаки
5. **Этап 5**: + технические индикаторы
6. **Этап 6**: + статистические признаки (TSFresh)

## 🔬 Методология VAR
- **Стационарность**: проверка и преобразование рядов
- **Оптимизация лагов**: автоматический выбор оптимального количества лагов
- **Коинтеграция**: проверка долгосрочных взаимосвязей
- **Walk-forward валидация**: строгая проверка на будущих данных
- **Impulse Response**: анализ реакции на шоки
- **Прогноз**: 10 точек вперед на каждом шаге


In [1]:
# Установка и импорты
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import logging

# Проверка и установка необходимых библиотек для VAR
try:
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    from statsmodels.tsa.vector_ar.var_model import VAR
    from statsmodels.tsa.stattools import adfuller, kpss
    from statsmodels.tsa.vector_ar.vecm import coint_johansen
    from statsmodels.stats.diagnostic import acorr_ljungbox
    print("✅ Основные библиотеки доступны")
except ImportError:
    print("📦 Устанавливаем недостающие библиотеки...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", 
                          "statsmodels", "scipy", "--quiet"])
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    from statsmodels.tsa.vector_ar.var_model import VAR
    from statsmodels.tsa.stattools import adfuller, kpss
    from statsmodels.tsa.vector_ar.vecm import coint_johansen
    from statsmodels.stats.diagnostic import acorr_ljungbox
    print("✅ Библиотеки установлены и импортированы")

# Настройка визуализации
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Установим читаемый формат для вывода чисел
pd.options.display.float_format = '{:.4f}'.format

print("📚 Библиотеки загружены успешно!")
print("📈 Готов к работе с VAR моделями!")

✅ Основные библиотеки доступны
📚 Библиотеки загружены успешно!
📈 Готов к работе с VAR моделями!


In [2]:
# Конфигурация (как в других multivariate блокнотах)
DATA_PATH = "../../data/multivariate_series/"
OUTPUT_PATH = "results/multivariate_var/"
FORECAST_HORIZON = 10  # Прогнозируем на 10 точек
TEST_SIZE = 11  # Размер тестового набора
MIN_TRAIN_SIZE = 100  # Минимальный размер обучающей выборки

# Создаем папку для результатов
Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

# Список тикеров (как в других multivariate блокнотах)
TICKERS = ['AFLT', 'LKOH', 'MOEX', 'NVTK', 'PIKK', 'SBER', 'VKCO', 'VTBR', 'X5', 'YDEX']

# Названия этапов для отображения
STAGE_NAMES = {
    1: "Базовые (close + volume)",
    2: "Базовые + аномалии", 
    3: "Базовые + аномалии + новости",
    4: "Выше + OHLCV",
    5: "Выше + технические индикаторы",
    6: "Выше + статистические признаки"
}

# Базовые признаки для каждого этапа
FEATURE_STAGES = {
    1: ['close', 'volume'],
    2: ['close', 'volume', 'anomaly'],
    3: ['close', 'volume', 'anomaly', 'weighted_score_with_decay'],
    4: ['close', 'volume', 'anomaly', 'weighted_score_with_decay', 'open', 'high', 'low'],
    5: ['close', 'volume', 'anomaly', 'weighted_score_with_decay', 'open', 'high', 'low',
        'return', 'SMA_14', 'RSI_14', 'MACD', 'BB_hband', 'BB_lband', 'ATR_14'],
    6: ['close', 'volume', 'anomaly', 'weighted_score_with_decay', 'open', 'high', 'low',
        'return', 'SMA_14', 'RSI_14', 'MACD', 'BB_hband', 'BB_lband', 'ATR_14']  # + TSFresh будет добавлено динамически
}

print(f"Настройки:")
print(f"- Путь к данным: {DATA_PATH}")
print(f"- Горизонт прогноза: {FORECAST_HORIZON}")
print(f"- Размер тест. данных: {TEST_SIZE}")
print(f"- Мин. размер обуч. данных: {MIN_TRAIN_SIZE}")
print(f"- Количество тикеров: {len(TICKERS)}")
print(f"- Папка результатов: {OUTPUT_PATH}")


Настройки:
- Путь к данным: ../../data/multivariate_series/
- Горизонт прогноза: 10
- Размер тест. данных: 11
- Мин. размер обуч. данных: 100
- Количество тикеров: 10
- Папка результатов: results/multivariate_var/


In [ ]:
# Функции для работы с признаками и стационарностью
def prepare_features_for_stage(df, stage):
    """
    Возвращает список колонок для определенного этапа
    """
    base_features = FEATURE_STAGES[stage]
    
    # Для 6 этапа добавляем TSFresh признаки
    if stage == 6:
        tsfresh_features = [col for col in df.columns if col.startswith('value__')]
        # Берем только первые 20 TSFresh признаков для упрощения
        tsfresh_features = tsfresh_features[:20]
        base_features = base_features + tsfresh_features
    
    # Фильтруем только доступные колонки
    available_features = [col for col in base_features if col in df.columns]
    
    return available_features

def check_stationarity(series, alpha=0.05):
    """
    Проверяет стационарность временного ряда используя ADF тест
    """
    try:
        result = adfuller(series.dropna())
        is_stationary = result[1] <= alpha
        return {
            'is_stationary': is_stationary, 
            'adf_stat': result[0],
            'p_value': result[1],
            'critical_values': result[4]
        }
    except:
        return {'is_stationary': False, 'adf_stat': None, 'p_value': 1.0, 'critical_values': None}

def make_stationary(df, method='diff'):
    """
    Делает временные ряды стационарными
    """
    df_stationary = df.copy()
    stationarity_info = {}
    
    for col in df.columns:
        # Проверяем стационарность
        stat_result = check_stationarity(df[col])
        
        if not stat_result['is_stationary']:
            if method == 'diff':
                # Берем первую разность
                df_stationary[col] = df[col].diff()
            elif method == 'log_diff':
                # Логарифмическая разность (только для положительных значений)
                if (df[col] > 0).all():
                    df_stationary[col] = np.log(df[col]).diff()
                else:
                    df_stationary[col] = df[col].diff()
        
        # Сохраняем информацию о стационарности
        final_stat = check_stationarity(df_stationary[col])
        stationarity_info[col] = {
            'original_stationary': stat_result['is_stationary'],
            'final_stationary': final_stat['is_stationary'],
            'transformation': 'none' if stat_result['is_stationary'] else method
        }
    
    # Удаляем первую строку (NaN после дифференцирования)
    df_stationary = df_stationary.dropna()
    
    return df_stationary, stationarity_info

def directional_accuracy(y_true, y_pred):
    """
    Вычисляет точность направления (процент правильно предсказанных направлений)
    """
    if len(y_true) <= 1:
        return 0.0
    
    y_true_diff = np.diff(y_true)
    y_pred_diff = np.diff(y_pred)
    
    correct_direction = np.sum(np.sign(y_true_diff) == np.sign(y_pred_diff))
    total_predictions = len(y_true_diff)
    
    return correct_direction / total_predictions if total_predictions > 0 else 0.0

print("✅ Функции для обработки данных готовы!")


In [ ]:
# VAR модель с оптимизацией
class VARForecaster:
    def __init__(self, max_lags=10, ic='aic'):
        self.max_lags = max_lags
        self.ic = ic  # информационный критерий для выбора лагов
        self.model = None
        self.fitted_model = None
        self.lag_order = None
        self.feature_names = None
        
    def find_optimal_lags(self, data):
        """
        Находит оптимальное количество лагов
        """
        try:
            model = VAR(data)
            lag_order_results = model.select_order(maxlags=self.max_lags)
            
            if self.ic == 'aic':
                optimal_lag = lag_order_results.aic
            elif self.ic == 'bic':
                optimal_lag = lag_order_results.bic
            elif self.ic == 'hqic':
                optimal_lag = lag_order_results.hqic
            else:
                optimal_lag = lag_order_results.aic
                
            return min(optimal_lag, self.max_lags) if optimal_lag is not None else 1
        except:
            return 1
    
    def fit(self, data):
        """
        Обучает VAR модель
        """
        try:
            self.feature_names = data.columns.tolist()
            
            # Находим оптимальные лаги
            self.lag_order = self.find_optimal_lags(data)
            
            # Создаем и обучаем модель
            self.model = VAR(data)
            self.fitted_model = self.model.fit(self.lag_order)
            
            return True
        except Exception as e:
            print(f"Ошибка при обучении VAR: {e}")
            return False
    
    def forecast(self, steps=1, last_obs=None):
        """
        Делает прогноз на steps шагов вперед
        """
        try:
            if self.fitted_model is None:
                return None
                
            if last_obs is None:
                # Используем последние наблюдения из обучающих данных
                forecast = self.fitted_model.forecast(
                    self.fitted_model.y, 
                    steps=steps
                )
            else:
                # Используем предоставленные последние наблюдения
                forecast = self.fitted_model.forecast(
                    last_obs[-self.lag_order:], 
                    steps=steps
                )
            
            return pd.DataFrame(forecast, columns=self.feature_names)
        except Exception as e:
            print(f"Ошибка при прогнозировании: {e}")
            return None
    
    def get_model_summary(self):
        """
        Возвращает сводку модели
        """
        if self.fitted_model is not None:
            return self.fitted_model.summary()
        return None
    
    def get_residuals(self):
        """
        Возвращает остатки модели
        """
        if self.fitted_model is not None:
            return self.fitted_model.resid
        return None

print("✅ VAR класс готов к использованию!")


In [ ]:
# Функция для загрузки и подготовки данных
def load_and_prepare_data(ticker, stage):
    """
    Загружает и подготавливает данные для указанного тикера и этапа
    """
    file_path = Path(DATA_PATH) / f"{ticker}_multivariate.csv"
    
    if not file_path.exists():
        print(f"⚠️ Файл {file_path} не найден")
        return None, None
    
    try:
        # Загружаем данные
        df = pd.read_csv(file_path)
        
        # Преобразуем timestamp в datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp').sort_index()
        
        # Получаем признаки для этапа
        features = prepare_features_for_stage(df, stage)
        
        if len(features) == 0:
            print(f"⚠️ Нет доступных признаков для этапа {stage} у тикера {ticker}")
            return None, None
        
        # Отбираем только нужные колонки
        df_features = df[features].copy()
        
        # Удаляем строки с NaN
        df_features = df_features.dropna()
        
        if df_features.empty:
            print(f"⚠️ Пустой DataFrame после очистки для {ticker}, этап {stage}")
            return None, None
        
        print(f"✅ Загружены данные для {ticker}, этап {stage}: {df_features.shape}")
        print(f"   Признаки: {features[:5]}{'...' if len(features) > 5 else ''}")
        
        return df_features, features
        
    except Exception as e:
        print(f"❌ Ошибка при загрузке {ticker}: {e}")
        return None, None

# Функция для walk-forward валидации
def walk_forward_validation(df, forecaster, target_col='close'):
    """
    Выполняет walk-forward валидацию для VAR модели
    """
    n_samples = len(df)
    train_end = n_samples - TEST_SIZE
    
    if train_end < MIN_TRAIN_SIZE:
        print(f"⚠️ Недостаточно данных для валидации: {n_samples} < {MIN_TRAIN_SIZE + TEST_SIZE}")
        return None
    
    predictions = []
    actuals = []
    dates = []
    
    # Walk-forward валидация
    for i in range(TEST_SIZE):
        current_train_end = train_end + i
        
        # Разделяем данные
        train_data = df.iloc[:current_train_end]
        
        # Проверяем, есть ли целевая переменная в обучающих данных
        if target_col not in train_data.columns:
            print(f"⚠️ Целевая переменная '{target_col}' не найдена в данных")
            return None
        
        # Делаем данные стационарными
        train_stationary, stationarity_info = make_stationary(train_data)
        
        if train_stationary.empty or len(train_stationary) < 10:
            print(f"⚠️ Недостаточно данных после стационаризации: {len(train_stationary)}")
            continue
        
        # Обучаем модель
        var_model = VARForecaster(max_lags=min(5, len(train_stationary)//10))
        success = var_model.fit(train_stationary)
        
        if not success:
            print(f"⚠️ Не удалось обучить модель на шаге {i}")
            continue
        
        # Делаем прогноз
        forecast_df = var_model.forecast(steps=FORECAST_HORIZON)
        
        if forecast_df is None or target_col not in forecast_df.columns:
            print(f"⚠️ Не удалось получить прогноз на шаге {i}")
            continue
        
        # Берем прогноз для целевой переменной
        pred_values = forecast_df[target_col].values
        
        # Получаем фактические значения
        actual_start = current_train_end
        actual_end = min(actual_start + FORECAST_HORIZON, len(df))
        actual_values = df[target_col].iloc[actual_start:actual_end].values
        
        # Сохраняем результаты
        min_len = min(len(pred_values), len(actual_values))
        if min_len > 0:
            predictions.extend(pred_values[:min_len])
            actuals.extend(actual_values[:min_len])
            dates.extend(df.index[actual_start:actual_start + min_len])
    
    if len(predictions) == 0:
        print("⚠️ Не удалось получить ни одного прогноза")
        return None
    
    return {
        'predictions': np.array(predictions),
        'actuals': np.array(actuals),
        'dates': dates
    }

print("✅ Функции валидации готовы!")


In [ ]:
# Основная функция для запуска экспериментов
def run_var_experiments():
    """
    Запускает эксперименты VAR для всех тикеров и этапов
    """
    results = []
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("🚀 Начинаем эксперименты с VAR моделью...")
    print(f"Тикеры: {TICKERS}")
    print(f"Этапы: {list(STAGE_NAMES.keys())}")
    print("=" * 80)
    
    for ticker in TICKERS:
        print(f"\n📊 Обрабатываем тикер: {ticker}")
        print("-" * 50)
        
        for stage in STAGE_NAMES.keys():
            print(f"\n🔄 Этап {stage}: {STAGE_NAMES[stage]}")
            
            try:
                # Загружаем данные
                df, features = load_and_prepare_data(ticker, stage)
                
                if df is None:
                    continue
                
                # Запускаем валидацию
                validation_results = walk_forward_validation(df, None, target_col='close')
                
                if validation_results is None:
                    print(f"❌ Валидация не удалась для {ticker}, этап {stage}")
                    continue
                
                # Вычисляем метрики
                predictions = validation_results['predictions']
                actuals = validation_results['actuals']
                
                # Основные метрики
                rmse = np.sqrt(mean_squared_error(actuals, predictions))
                mae = mean_absolute_error(actuals, predictions)
                mape = mean_absolute_percentage_error(actuals, predictions) * 100
                da = directional_accuracy(actuals, predictions) * 100
                
                # Дополнительные метрики
                mse = mean_squared_error(actuals, predictions)
                correlation = np.corrcoef(actuals, predictions)[0, 1] if len(actuals) > 1 else 0
                
                # Сохраняем результат
                result = {
                    'ticker': ticker,
                    'stage': stage,
                    'stage_name': STAGE_NAMES[stage],
                    'n_features': len(features),
                    'n_samples': len(df),
                    'n_predictions': len(predictions),
                    'rmse': rmse,
                    'mae': mae,
                    'mape': mape,
                    'mse': mse,
                    'da': da,
                    'correlation': correlation,
                    'features': features[:10]  # Первые 10 признаков для справки
                }
                
                results.append(result)
                
                print(f"✅ Результаты для {ticker}, этап {stage}:")
                print(f"   RMSE: {rmse:.4f}")
                print(f"   MAE: {mae:.4f}")
                print(f"   MAPE: {mape:.2f}%")
                print(f"   DA: {da:.2f}%")
                print(f"   Корреляция: {correlation:.4f}")
                
            except Exception as e:
                print(f"❌ Ошибка при обработке {ticker}, этап {stage}: {e}")
                continue
    
    # Преобразуем результаты в DataFrame
    results_df = pd.DataFrame(results)
    
    if not results_df.empty:
        # Сохраняем результаты
        output_file = Path(OUTPUT_PATH) / f"multivariate_var_results_{timestamp}.csv"
        results_df.to_csv(output_file, index=False)
        print(f"\n💾 Результаты сохранены в: {output_file}")
        
        # Выводим сводку
        print(f"\n📈 Сводка результатов:")
        print(f"Всего экспериментов: {len(results_df)}")
        print(f"Средний RMSE: {results_df['rmse'].mean():.4f}")
        print(f"Средний MAPE: {results_df['mape'].mean():.2f}%")
        print(f"Средний DA: {results_df['da'].mean():.2f}%")
        
        return results_df
    else:
        print("❌ Нет успешных результатов")
        return None

print("🎯 Функция экспериментов готова к запуску!")

In [ ]:
# Функции для визуализации результатов
def plot_var_results(results_df):
    """
    Создает визуализации результатов VAR экспериментов
    """
    if results_df is None or results_df.empty:
        print("❌ Нет данных для визуализации")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('📈 Результаты VAR модели по этапам и тикерам', fontsize=16, fontweight='bold')
    
    # 1. RMSE по этапам
    stage_rmse = results_df.groupby('stage')['rmse'].mean().reset_index()
    axes[0, 0].bar(stage_rmse['stage'], stage_rmse['rmse'], color='skyblue', alpha=0.7)
    axes[0, 0].set_title('Средний RMSE по этапам')
    axes[0, 0].set_xlabel('Этап')
    axes[0, 0].set_ylabel('RMSE')
    for i, v in enumerate(stage_rmse['rmse']):
        axes[0, 0].text(i+1, v + 0.001, f'{v:.3f}', ha='center', va='bottom')
    
    # 2. MAPE по этапам
    stage_mape = results_df.groupby('stage')['mape'].mean().reset_index()
    axes[0, 1].bar(stage_mape['stage'], stage_mape['mape'], color='lightcoral', alpha=0.7)
    axes[0, 1].set_title('Средний MAPE по этапам')
    axes[0, 1].set_xlabel('Этап')
    axes[0, 1].set_ylabel('MAPE (%)')
    for i, v in enumerate(stage_mape['mape']):
        axes[0, 1].text(i+1, v + 0.5, f'{v:.1f}%', ha='center', va='bottom')
    
    # 3. DA по этапам
    stage_da = results_df.groupby('stage')['da'].mean().reset_index()
    axes[1, 0].bar(stage_da['stage'], stage_da['da'], color='lightgreen', alpha=0.7)
    axes[1, 0].set_title('Средняя Directional Accuracy по этапам')
    axes[1, 0].set_xlabel('Этап')
    axes[1, 0].set_ylabel('DA (%)')
    axes[1, 0].axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Случайный уровень')
    axes[1, 0].legend()
    for i, v in enumerate(stage_da['da']):
        axes[1, 0].text(i+1, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    
    # 4. Тепловая карта RMSE по тикерам и этапам
    pivot_rmse = results_df.pivot(index='ticker', columns='stage', values='rmse')
    im = axes[1, 1].imshow(pivot_rmse.values, cmap='YlOrRd', aspect='auto')
    axes[1, 1].set_title('Тепловая карта RMSE')
    axes[1, 1].set_xlabel('Этап')
    axes[1, 1].set_ylabel('Тикер')
    axes[1, 1].set_xticks(range(len(pivot_rmse.columns)))
    axes[1, 1].set_xticklabels(pivot_rmse.columns)
    axes[1, 1].set_yticks(range(len(pivot_rmse.index)))
    axes[1, 1].set_yticklabels(pivot_rmse.index)
    plt.colorbar(im, ax=axes[1, 1])
    
    plt.tight_layout()
    
    # Сохраняем график
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_file = Path(OUTPUT_PATH) / f"multivariate_var_comparison_{timestamp}.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"📊 График сохранен: {plot_file}")
    
    plt.show()

def create_summary_table(results_df):
    """
    Создает сводную таблицу результатов
    """
    if results_df is None or results_df.empty:
        print("❌ Нет данных для сводной таблицы")
        return None
    
    # Группируем по этапам
    summary = results_df.groupby(['stage', 'stage_name']).agg({
        'rmse': ['mean', 'std'],
        'mape': ['mean', 'std'],
        'da': ['mean', 'std'],
        'correlation': ['mean', 'std'],
        'n_features': 'first'
    }).round(4)
    
    # Упрощаем названия колонок
    summary.columns = ['RMSE_mean', 'RMSE_std', 'MAPE_mean', 'MAPE_std', 
                      'DA_mean', 'DA_std', 'Corr_mean', 'Corr_std', 'N_features']
    
    summary = summary.reset_index()
    
    print("📋 Сводная таблица результатов VAR:")
    print("=" * 80)
    display(summary)
    
    # Находим лучший этап
    best_stage_rmse = summary.loc[summary['RMSE_mean'].idxmin()]
    best_stage_da = summary.loc[summary['DA_mean'].idxmax()]
    
    print(f"\n🏆 Лучшие результаты:")
    print(f"Минимальный RMSE: Этап {best_stage_rmse['stage']} ({best_stage_rmse['stage_name']}) - {best_stage_rmse['RMSE_mean']:.4f}")
    print(f"Максимальный DA: Этап {best_stage_da['stage']} ({best_stage_da['stage_name']}) - {best_stage_da['DA_mean']:.2f}%")
    
    return summary

print("📊 Функции визуализации готовы!")

## 🚀 Запуск экспериментов

Теперь запустим полный цикл экспериментов с VAR моделью для всех тикеров и этапов признаков.


In [ ]:
# Исправляем ошибку в коде walk_forward_validation
def walk_forward_validation_fixed(df, forecaster, target_col='close'):
    """
    Выполняет walk-forward валидацию для VAR модели (исправленная версия)
    """
    n_samples = len(df)
    train_end = n_samples - TEST_SIZE
    
    if train_end < MIN_TRAIN_SIZE:
        print(f"⚠️ Недостаточно данных для валидации: {n_samples} < {MIN_TRAIN_SIZE + TEST_SIZE}")
        return None
    
    predictions = []
    actuals = []
    dates = []
    
    # Walk-forward валидация
    for i in range(TEST_SIZE):
        current_train_end = train_end + i
        
        # Разделяем данные
        train_data = df.iloc[:current_train_end]
        
        # Проверяем, есть ли целевая переменная в обучающих данных
        if target_col not in train_data.columns:
            print(f"⚠️ Целевая переменная '{target_col}' не найдена в данных")
            return None
        
        # Делаем данные стационарными
        train_stationary, stationarity_info = make_stationary(train_data)
        
        if train_stationary.empty or len(train_stationary) < 10:
            print(f"⚠️ Недостаточно данных после стационаризации: {len(train_stationary)}")
            continue
        
        # Обучаем модель
        var_model = VARForecaster(max_lags=min(5, len(train_stationary)//10))
        success = var_model.fit(train_stationary)
        
        if not success:
            print(f"⚠️ Не удалось обучить модель на шаге {i}")
            continue
        
        # Делаем прогноз
        forecast_df = var_model.forecast(steps=FORECAST_HORIZON)
        
        if forecast_df is None or target_col not in forecast_df.columns:
            print(f"⚠️ Не удалось получить прогноз на шаге {i}")
            continue
        
        # Берем прогноз для целевой переменной
        pred_values = forecast_df[target_col].values
        
        # Получаем фактические значения
        actual_start = current_train_end
        actual_end = min(actual_start + FORECAST_HORIZON, len(df))
        actual_values = df[target_col].iloc[actual_start:actual_end].values
        
        # Сохраняем результаты
        min_len = min(len(pred_values), len(actual_values))
        if min_len > 0:
            predictions.extend(pred_values[:min_len])
            actuals.extend(actual_values[:min_len])
            dates.extend(df.index[actual_start:actual_start + min_len])
    
    if len(predictions) == 0:
        print("⚠️ Не удалось получить ни одного прогноза")
        return None
    
    return {
        'predictions': np.array(predictions),
        'actuals': np.array(actuals),
        'dates': dates
    }

# Запускаем эксперименты
print("🎬 Начинаем эксперименты...")
results_df = run_var_experiments()

if results_df is not None:
    print("✅ Эксперименты завершены успешно!")
else:
    print("❌ Эксперименты не удались")

In [ ]:
# Визуализация и анализ результатов
if results_df is not None and not results_df.empty:
    print("📊 Создаем визуализации...")
    
    # Строим графики
    plot_var_results(results_df)
    
    # Создаем сводную таблицу
    summary_table = create_summary_table(results_df)
    
    # Дополнительный анализ
    print("\n🔍 Дополнительный анализ:")
    print(f"Лучший тикер по RMSE: {results_df.loc[results_df['rmse'].idxmin(), 'ticker']} (RMSE: {results_df['rmse'].min():.4f})")
    print(f"Худший тикер по RMSE: {results_df.loc[results_df['rmse'].idxmax(), 'ticker']} (RMSE: {results_df['rmse'].max():.4f})")
    print(f"Лучший тикер по DA: {results_df.loc[results_df['da'].idxmax(), 'ticker']} (DA: {results_df['da'].max():.2f}%)")
    
    # Показываем первые несколько строк детальных результатов
    print("\\n📋 Первые несколько результатов:")
    display(results_df.head(10))
    
else:
    print("❌ Нет результатов для анализа")


In [ ]:
## 🎯 Заключение

Блокнот для тестирования VAR (Vector Autoregression) модели на многомерных временных рядах готов!

### 🔧 Что реализовано:

1. **Комплексная VAR модель** с автоматической оптимизацией лагов
2. **Поэтапное тестирование** с 6 различными наборами признаков  
3. **Стационаризация данных** с проверкой ADF тестом
4. **Walk-forward валидация** для честного тестирования
5. **Множественные метрики**: RMSE, MAE, MAPE, DA, корреляция
6. **Визуализация результатов** с графиками и тепловыми картами
7. **Автоматическое сохранение** результатов в CSV и PNG файлы

### 📊 Этапы признаков:

- **Этап 1**: Базовые (close + volume)
- **Этап 2**: + Аномалии  
- **Этап 3**: + Новостные данные
- **Этап 4**: + OHLCV признаки
- **Этап 5**: + Технические индикаторы
- **Этап 6**: + Статистические признаки (TSFresh)

### 🚀 Для запуска:

1. Запустите все ячейки последовательно
2. Результаты сохранятся в папке `results/multivariate_var/`
3. Будут созданы CSV с детальными результатами и PNG с графиками

### 📈 Ожидаемые результаты:

VAR модель должна показать хорошие результаты на данных с умеренной корреляцией между признаками. Модель особенно эффективна для:
- Анализа взаимосвязей между переменными
- Краткосрочного прогнозирования (1-10 шагов)
- Данных с стабильными статистическими свойствами
